In [43]:
def crop_img(img, threshold=100):
    '''
    Crop the image to get the region of interest. Remove the vignette frame.
    Analyze the value of the pixels in the diagonal of the image, from 0,0 to h,w and
    take the points where this value crosses the threshold by the first time and for last.
    Args:
    - img (numpy ndarray): Image to crop.
    - threshold (int): Value to split the diagonal into image and frame.
    Return:
    - The coordinates of the rectangle and the cropped image.
    '''
    # Get the image dimensions
    h, w = img.shape[:2]

    # Get the coordinates of the pixels in the diagonal
    if h != 1024:
        y_coords = ([i for i in range(0, h, 3)], [i for i in range(h - 3, -1, -3)])
    y_coords = ([i for i in range(0, h, 4)], [i for i in range(h - 4, -1, -4)])
    x_coords = ([i for i in range(0, w, 4)], [i for i in range(0, w, 4)])

    # Get the mean value of the pixels in the diagonal, form 0,0 to h,w
    # and from h,0 to 0,w
    coordinates = {'y1_1': 0, 'x1_1': 0, 'y2_1': h, 'x2_1': w, 'y1_2': h, 'x1_2': 0, 'y2_2': 0, 'x2_2': w}
    for i in range(2):
        d = []
        y1_aux, x1_aux = 0, 0
        y2_aux, x2_aux = h, w
        for y, x in zip(y_coords[i], x_coords[i]):
            d.append(np.mean(img[y, x, :]))

        # Get the location of the first point where the threshold is crossed
        for idx, value in enumerate(d):
            if value >= threshold:
                coordinates['y1_' + str(i + 1)] = y_coords[i][idx]
                coordinates['x1_' + str(i + 1)] = x_coords[i][idx]
                break

        # Get the location of the last point where the threshold is crossed
        for idx, value in enumerate(reversed(d)):
            if value >= threshold:
                coordinates['y2_' + str(i + 1)] = y_coords[i][-idx if idx != 0 else -1]
                coordinates['x2_' + str(i + 1)] = x_coords[i][-idx if idx != 0 else -1]
                break

    # Set the coordinates to crop the image

    # # modified
    # # y1 = min(coordinates['y1_1'], coordinates['y1_2'])
    # # y2 = max(coordinates['y2_1'], coordinates['y2_2'])

    # # original
    # y1 = max(coordinates['y1_1'], coordinates['y2_2'])
    # y2 = min(coordinates['y2_1'], coordinates['y1_2'])

    # # original
    # # x1 = max(coordinates['x1_1'], coordinates['x1_2'])
    # # x2 = min(coordinates['x2_1'], coordinates['x2_2'])

    # # modified
    # x1 = max(coordinates['x1_1'], coordinates['x2_2'])
    # x2 = min(coordinates['x2_1'], coordinates['x1_2'])

    # Set the coordinates to crop the image
    y1 = max(coordinates['y1_1'], coordinates['y2_2'])
    y2 = min(coordinates['y2_1'], coordinates['y1_2'])
    x1 = max(coordinates['x1_1'], coordinates['x1_2'])
    x2 = min(coordinates['x2_1'], coordinates['x2_2'])

    return img[y1:y2, x1:x2, :], y1, y2, x1, x2

In [18]:
# Load Libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from dataset.dataset import SkinLesion_Dataset, SegExamples

# Load Dataset
data = SegExamples()
data[0].keys()
print(data[1]['type'])
# print(examples.seg_examples_df)
# print(data.seg_examples_df.keys())
# plt.imshow(examples[1]['img'])
# print(examples[1]['type'])

easy


In [22]:
pwd

'C:\\Users\\Lenovo\\PycharmProjects\\Skin_lesion_segmentation\\notebooks'

In [ ]:
def hair_removal(paths_list):
    # red channel used. after mask creation, opening + dilation is applied to extend the mask.
    count = 0
    for i in paths_list:
        p = "data/"+i
        image = cv2.imread(p, cv2.IMREAD_COLOR)
        # red_channel
        red = image[:,:,2]

        # Gaussian filter
        gaussian= cv2.GaussianBlur(red,(3,3),cv2.BORDER_DEFAULT)

        # Black hat filter
        kernel = cv2.getStructuringElement(1,(20,20))
        blackhat = cv2.morphologyEx(gaussian, cv2.MORPH_BLACKHAT, kernel)

        #Binary thresholding (MASK)
        ret,mask = cv2.threshold(blackhat,10,255,cv2.THRESH_BINARY)

        # apply opening : erosion+dilation (remove the dots that are captured and then extend the hair parts)
        kernel_opening = np.ones((1, 1), np.uint8)
        opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel_opening)
        kernel_dilation = np.ones((5, 5), np.uint8)
        dilated_mask = cv2.dilate(opening, kernel_dilation, iterations=1)

        #Replace pixels of the mask
        dst = cv2.inpaint(image,dilated_mask,6,cv2.INPAINT_TELEA)

        count+=1


In [48]:
# Hir remova + K-means Segmentation
paths = "../data/"+data.seg_examples_df.path
count = 0
for i in paths:
    type = data[count]['type']
    img = cv2.imread(i)

    # Hair removal

    red = img[:,:,2]
    # Gaussian filter
    gaussian= cv2.GaussianBlur(red,(3,3),cv2.BORDER_DEFAULT)
    # Black hat filter
    kernel = cv2.getStructuringElement(1,(20,20))
    blackhat = cv2.morphologyEx(gaussian, cv2.MORPH_BLACKHAT, kernel)
    #Binary thresholding (MASK)
    ret,mask = cv2.threshold(blackhat,10,255,cv2.THRESH_BINARY)
    # apply opening : erosion+dilation (remove the dots that are captured and then extend the hair parts)
    kernel_opening = np.ones((1, 1), np.uint8)
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel_opening)
    kernel_dilation = np.ones((5, 5), np.uint8)
    dilated_mask = cv2.dilate(opening, kernel_dilation, iterations=1)
    #Replace pixels of the mask
    dst = cv2.inpaint(img,dilated_mask,6,cv2.INPAINT_TELEA)

    ## Border removal
    image_cropped, y1, y2, x1, x2 = crop_img(dst)

    # kmeans
    image = cv2.cvtColor(image_cropped, cv2.COLOR_BGR2GRAY)
    pixel_values = image.reshape(-1)
    pixel_values = np.float32(pixel_values)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    k = 2
    _, labels, (centers) = cv2.kmeans(pixel_values, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    labels = labels.flatten()
    segmented_image = centers[labels.flatten()]
    # reshape back to the original image dimension
    segmented_image = segmented_image.reshape(image.shape)
    ret1,seg_bin = cv2.threshold(segmented_image,127,255,cv2.THRESH_BINARY_INV)
    # Save the image
    count=count+1
    cv2.imwrite(f'../examples/kmeans2/{type}_{count}.png', seg_bin)

    # show the image
    # plt.imshow(segmented_image, cmap="gray")
    # plt.show()


KeyboardInterrupt: 

In [ ]:
# Base k meeans code
paths = "../data/"+data.seg_examples_df.path
count = 0
for i in paths:
    type = data[count]['type']
    img = cv2.imread(i)
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pixel_values = image.reshape(-1,3)
    pixel_values = np.float32(pixel_values)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    k = 2
    _, labels, (centers) = cv2.kmeans(pixel_values, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    labels = labels.flatten()
    segmented_image = centers[labels.flatten()]
    # reshape back to the original image dimension
    segmented_image = segmented_image.reshape(image.shape)
    # Save the image
    count=count+1
    cv2.imwrite(f'../examples/kmeans/{type}_{count}.png', segmented_image)

    # show the image
    plt.imshow(segmented_image, cmap="gray")
    plt.show()

## Feature Extraction

In [ ]:
# TASKS
# from the segmented image:
# the color channel r,g,b = color histogram
# mean, mode, std, skewness, energy, entropy, kurtosis
# min and max values for each channel.
# get the hsv space color info and calculate the same mean, ... for it too.
# glcm - contrast, correlation, enegry, homogenity

# extract local and global segmented image features and then compare them.

In [49]:
# Color histogram statistics
 data[1]['img']

array([[[0, 0, 0],
        [1, 1, 1],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[1, 1, 1],
        [0, 0, 0],
        [1, 1, 1],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[1, 1, 1],
        [0, 0, 0],
        [1, 1, 1],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [1, 1, 1],
        [1, 1, 1],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)